In [1]:
import keras
import numpy as np
import h5py
import math
import gc
import os
import tensorflow as tf
from keras import layers,optimizers
from keras.preprocessing import image
from keras.models import Model, load_model,Sequential
from keras.layers import  Concatenate,add, Dropout,Conv3D
from keras.layers import Input, Add, Dense, Activation, Flatten, Conv2D, MaxPooling2D,multiply,Lambda
from keras.callbacks import ModelCheckpoint,History,ReduceLROnPlateau
import keras.backend.tensorflow_backend as K

os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2" # 使用第几个GPU， 0是第一个
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.
/home/wangchong/.conda/envs/tensorflow114/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wangchong/.conda/envs/tensorflow114/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wangchong/.conda/envs/tensorflow114/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [2]:
def max_dim_4d(tensor):
    return K.max(tensor, axis=4)

def mean_dim_4d(tensor):
    return K.mean(tensor, axis=4)

def expand_dim_4d(tensor):
    return K.expand_dims(tensor, axis=4)

def attention_spatial_4d(x,Channel_Num):       
    x1 = Lambda(max_dim_4d)(x)
    x2 = Lambda(mean_dim_4d)(x)      
    x11=Lambda(expand_dim_4d)(x1)
    x22=Lambda(expand_dim_4d)(x2)
    f=concatenate([x11,x22],4)
    f1 = Conv3D(1, kernel_size= [5, 5, 5], strides=(1, 1, 1), padding= 'same',data_format='channels_last', 
                    dilation_rate=(1, 1, 1), activation=keras.activations.relu,use_bias= True )(inputs)
    rate = Activation('sigmoid')(f1)
    atts_x = multiply([x, rate])
    return atts_x

def max_dimc(tensor):
    return K.max(tensor, axis=1)

def mean_dimc(tensor):
    return K.mean(tensor, axis=1)

def attention_channel_3d(x,Channel_Num):
    x1 = Lambda(max_dimc)(x)
    x11 = Lambda(max_dimc)(x1)
    x2 = Lambda(mean_dimc)(x)
    x22 = Lambda(mean_dimc)(x2)
    x111   = Dense(int(Channel_Num/2), activation = 'relu')(x11)
    x1111   = Dense(Channel_Num, activation = 'sigmoid')(x111)
    x222   = Dense(int(Channel_Num/2), activation = 'relu')(x22)
    x2222   = Dense(Channel_Num, activation = 'sigmoid')(x222)    
    attc0_x = add([x1111,x2222])
    attc_x = multiply([x, attc0_x])
    return attc_x

def max_dimc_4d1(tensor):
    return K.max(tensor, axis=1)

def mean_dimc_4d1(tensor):
    return K.mean(tensor, axis=1)

def attention_channel_4d(x,Channel_Num):
    x1 = Lambda(max_dimc_4d1)(x)
    x11 = Lambda(max_dimc_4d1)(x1)
    x11 = Lambda(max_dimc_4d1)(x11)
    x2 = Lambda(mean_dimc_4d1)(x)
    x22 = Lambda(mean_dimc_4d1)(x2)
    x22 = Lambda(mean_dimc_4d1)(x22)
    x111   = Dense(int(Channel_Num*2), activation = 'relu')(x11)
    x1111   = Dense(Channel_Num, activation = 'sigmoid')(x111)
    x222   = Dense(int(Channel_Num*2), activation = 'relu')(x22)
    x2222   = Dense(Channel_Num, activation = 'sigmoid')(x222)    
    attc0_x = add([x1111,x2222])
    attc_x = multiply([x, attc0_x])
    return attc_x

def VGG192d(input_shape=(224, 224, 5), classes=15, filters=[64, 128, 256, 512, 1024]):
    F1, F2, F3, F4, F5 = filters
    inputs = keras.Input(shape = input_shape)
    conv1_1 = keras.layers.Conv2D(F1, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(inputs)
    conv1_2 = keras.layers.Conv2D(F1, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv1_1)
    pooling1 = keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2], padding= 'same')(conv1_2)
    conv2_1 = keras.layers.Conv2D(F2, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(pooling1)
    conv2_2 = keras.layers.Conv2D(F2, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv2_1)
    pooling2 = keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2], padding= 'same')(conv2_2)
    conv3_1 = keras.layers.Conv2D(F3, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(pooling2)
    conv3_2 = keras.layers.Conv2D(F3, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv3_1)
    conv3_3 = keras.layers.Conv2D(F3, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv3_2)
    conv3_4 = keras.layers.Conv2D(F3, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv3_3)
    pooling3 = keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2], padding= 'same')(conv3_4)
    conv4_1 = keras.layers.Conv2D(F4, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(pooling3)
    conv4_2 = keras.layers.Conv2D(F4, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv4_1)
    conv4_3 = keras.layers.Conv2D(F4, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv4_2)
    conv4_4 = keras.layers.Conv2D(F4, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv4_3)
    pooling4 = keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2], padding= 'same')(conv4_4)
    conv5_1 = keras.layers.Conv2D(F5, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(pooling4)
    conv5_2 = keras.layers.Conv2D(F5, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv5_1)
    conv5_3 = keras.layers.Conv2D(F5, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv5_2)
    conv5_4 = keras.layers.Conv2D(F5, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(conv5_3)
    pooling5 = keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2], padding= 'same')(conv5_4)   
    flatten = keras.layers.Flatten()(pooling5)
    fc1 = keras.layers.Dense(1024, activation=keras.activations.relu, use_bias= True)(flatten)
    fc1 = keras.layers.Dense(1024, activation=keras.activations.relu, use_bias= True)(fc1)
    prediction = keras.layers.Dense(classes, activation=keras.activations.relu,use_bias= True)(fc1)
    model = keras.Model(inputs= inputs, outputs = prediction)
    return model

def F3d(x,filters):
    F1, F2, F3 = filters    
    x = keras.layers.Conv3D(F1, kernel_size= [3, 3, 5], strides=(1, 1, 1), padding= 'same',data_format='channels_last',
                    dilation_rate=(1, 1, 1), activation=keras.activations.relu,use_bias= True)(x)
    x = keras.layers.Conv3D(F2, kernel_size= [3, 3, 5], strides=(1, 1, 1), padding= 'same',data_format='channels_last',
                    dilation_rate=(1, 1, 1), activation=keras.activations.relu,use_bias= True)(x)
    x = keras.layers.Conv3D(F3, kernel_size= [3, 3, 5], strides=(1, 1, 1), padding= 'same',data_format='channels_last',
                    dilation_rate=(1, 1, 1), activation=keras.activations.relu,use_bias= True)(x)
    x = keras.layers.MaxPooling3D(pool_size= [2, 2, 2], strides= [2, 2, 1], padding= 'same',data_format='channels_last')(x)
    return x

def F2d(X,filters):
    F1, F2, F3 = filters
    X  = keras.layers.Conv2D(F1, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(X)
    X = keras.layers.Conv2D(F2, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(X)
    X = keras.layers.Conv2D(F3, kernel_size= [3, 3], strides= [1, 1], activation= keras.activations.relu, use_bias= True, padding= 'same')(X)
    X = keras.layers.MaxPooling2D(pool_size= [3, 3], strides= [2, 2], padding= 'same')(X)
    return X

In [3]:
def TCIF_fusion():    
    inputs_u  = keras.Input(shape = (25, 25, 5, 4))
    inputs_v  = keras.Input(shape = (25, 25, 5, 4))
    inputs_w  = keras.Input(shape = (25, 25, 5, 4))
    inputs_sst  = keras.Input(shape = (25, 25, 5, 1))
    inputs_h  = keras.Input(shape = (25, 25, 85))

    X_U = F3d(inputs_u,filters=[128, 128, 128])
    X_V = F3d(inputs_v,filters=[128, 128, 128])
    X_W = F3d(inputs_w,filters=[128, 128, 128])
    X_T = F3d(inputs_sst,filters=[128, 128, 128])
    con = add([X_U,X_V,X_W,X_T])
   
    X_U = F3d(X_U,filters=[256, 256, 256])
    X_V = F3d(X_V,filters=[256, 256, 256])
    X_W = F3d(X_W,filters=[256, 256, 256])
    X_T = F3d(X_T,filters=[256, 256, 256])
    X_C = F3d(con,filters=[256, 256, 256])
    con = add([X_U,X_V,X_W,X_T,X_C])

    
    X_U = F3d(X_U,filters=[512, 512, 512])
    X_V = F3d(X_V,filters=[512, 512, 512])
    X_W = F3d(X_W,filters=[512, 512, 512])
    X_T = F3d(X_T,filters=[512, 512, 512])
    X_C = F3d(con,filters=[512, 512, 512])
    con = add([X_U,X_V,X_W,X_T,X_C])

    X_H = F2d(inputs_h,filters=[128, 128, 128])
    X_H = F2d(X_H,filters=[256, 256, 256])
    X_H = F2d(X_H,filters=[512, 512, 512])

    X_U = Flatten()(X_U)
    X_V = Flatten()(X_V)
    X_W = Flatten()(X_W)
    X_T = Flatten()(X_T)
    X_C = Flatten()(con)
    X_H = Flatten()(X_H)

    con = Concatenate()([X_U,X_V,X_W,X_T,X_C,X_H])
    con = keras.layers.Dense(256, activation=keras.activations.relu,use_bias= True)(con)
    con = keras.layers.Dense(15, activation=keras.activations.relu,use_bias= True)(con)

    left_input = Input(shape=[30])
    fc1_l   = Dense(32, activation = 'relu')(left_input)
    fc1_l   = Dense(64, activation = 'relu')(fc1_l)

    model_sat = VGG192d(input_shape=(224, 224, 5), classes=15,filters=[64, 128, 256, 512, 512])
    #Dropout_Rate=0.3
    con = Concatenate()([con,fc1_l,model_sat.output])
    #con = Dropout(Dropout_Rate)(con)
    X1  = keras.layers.Dense(256, activation=keras.activations.relu,use_bias= True)(con)
    X   = keras.layers.Dense(256, activation=keras.activations.relu,use_bias= True)(X1)
    X   = keras.layers.Dense(256, activation=keras.activations.relu,use_bias= True)(X)
    con = add([X,X1])
    #con = Dropout(Dropout_Rate)(con)
    X1  = keras.layers.Dense(128, activation=keras.activations.relu,use_bias= True)(con)
    X   = keras.layers.Dense(128, activation=keras.activations.relu,use_bias= True)(X1)
    X   = keras.layers.Dense(128, activation=keras.activations.relu,use_bias= True)(X)
    con = add([X,X1])
    #con = Dropout(Dropout_Rate)(con)
    predictions = keras.layers.Dense(1, activation= 'linear', use_bias= True)(con)

    model = Model(inputs=[inputs_u,inputs_v,inputs_w,inputs_sst,inputs_h,left_input,model_sat.input], outputs=predictions)# 
    return model

model=TCIF_fusion()
model.summary()





__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 5)  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 224, 224, 64) 2944        input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 224, 224, 64) 36928       conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 112, 112, 64) 0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d

In [ ]:
# An example of obtaining model knowledge from the train_sst
for i in range(len(train_u)):
    if np.mod(i,50)==0:
        K.clear_session()
        model=load_model('./CMA/TCIF-fusion-best-uvwssthsat-1.h5')
    aa=np.zeros(shape=(1,25,25,65))
    aa=train_h[i,:,:,:]
    aa=np.expand_dims(aa,axis=0)
    bb=np.zeros(shape=(1,25,25,5,4))
    bb=train_u[i,:,:,:]
    bb=np.expand_dims(bb,axis=0)
    cc=np.zeros(shape=(1,25,25,5,4))
    cc=train_v[i,:,:,:]
    cc=np.expand_dims(cc,axis=0)
    dd=np.zeros(shape=(1,25,25,5,4))
    dd=train_w[i,:,:,:]
    dd=np.expand_dims(dd,axis=0)
    ee=np.zeros(shape=(1,25,25,5,1))
    ee=train_sst[i,:,:,:]
    ee=np.expand_dims(ee,axis=0)
    ff=np.zeros(shape=(1,30))
    ff=train_left[i,:]
    ff=np.expand_dims(ff,axis=0)
    gg=np.zeros(shape=(1,224,224,5))
    gg=train_sst[i,:,:,:]
    gg=np.expand_dims(ee,axis=0)
    african_elephant_output = model.output[:, 0]
    last_conv_layer = model.get_layer('input_4')    
    grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input[0],model.input[1],model.input[2],model.input[3],
                     model.input[4],model.input[5],model.input[6]], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([bb,cc,dd,ee,aa,ff,gg])
    train_cma_sst[i,:,:,:]=conv_layer_output_value   
    del conv_layer_output_value